# 1. Import Library

In [52]:
# import library
import pandas as pd
import numpy as np
import seaborn as sns
import glob
import re
import nltk
import spacy
import emoji
import string
import matplotlib.pyplot as plt
from nltk.tokenize import word_tokenize
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

# 2. Memuat Dataset

In [53]:
# load data
data = pd.read_csv('data_komentar_youtube.csv')
data

,Username,Komentar,Timestamp,Likes,Original Text,Public
0,@blackberry1986,Coba dpr kalau mau buat undang&quot; bikin vot...,2025-03-25T03:07:32Z,0,"Coba dpr kalau mau buat undang"" bikin vote rak...",True
1,@UbaidillahAlhasani,mas adi dan bg pandji serti publik yg ada di k...,2025-03-19T21:29:18Z,0,mas adi dan bg pandji serti publik yg ada di k...,True
2,@BagasRyan-w5q,Kelas mas adi prayitno memang benar yg kalah g...,2025-03-17T19:06:15Z,0,Kelas mas adi prayitno memang benar yg kalah g...,True
3,@sahrulmeltas7551,Dari sini kita faham bahwa parpol nya tidak ma...,2025-03-14T07:38:22Z,0,Dari sini kita faham bahwa parpol nya tidak ma...,True
4,@SadegaTri,"Seneng lihat acara ini, tapi sedikit masih sda...",2025-03-09T18:48:53Z,0,"Seneng lihat acara ini, tapi sedikit masih sda...",True
...,...,...,...,...,...,...
18587,@whoami4767,Oh tempe,2025-03-22T00:54:53Z,5,Oh tempe,True
18588,@firmantuhan-k9o,"TIM BOCOR ALUS JANGAN TAKUT, UNTUK KEBENARAN T...",2025-03-22T00:34:47Z,46,"TIM BOCOR ALUS JANGAN TAKUT, UNTUK KEBENARAN T...",True
18589,@Local_indosuke2004,Presiden beban,2025-03-22T00:22:54Z,23,Presiden beban,True
18590,@andipamungkas3397,"Semangat terus tempo, jangan kalah sama teror ...",2025-03-21T23:39:37Z,96,"Semangat terus tempo, jangan kalah sama teror ...",True


# 3. Exploratory Data Analysis (EDA)

In [54]:
# informasi data
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18592 entries, 0 to 18591
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Username       18587 non-null  object
 1   Komentar       18592 non-null  object
 2   Timestamp      18592 non-null  object
 3   Likes          18592 non-null  int64 
 4   Original Text  18588 non-null  object
 5   Public         18592 non-null  bool  
dtypes: bool(1), int64(1), object(4)
memory usage: 744.5+ KB


In [55]:
# dataframe shape
data.shape

(18592, 6)

In [56]:
# melihat missing value
data.isnull().sum()

Username         5
Komentar         0
Timestamp        0
Likes            0
Original Text    4
Public           0
dtype: int64

In [57]:
# melihat data yang duplikat
data.duplicated().sum()

np.int64(8)

# 4. Data Preprocessing

In [58]:
data

,Username,Komentar,Timestamp,Likes,Original Text,Public
0,@blackberry1986,Coba dpr kalau mau buat undang&quot; bikin vot...,2025-03-25T03:07:32Z,0,"Coba dpr kalau mau buat undang"" bikin vote rak...",True
1,@UbaidillahAlhasani,mas adi dan bg pandji serti publik yg ada di k...,2025-03-19T21:29:18Z,0,mas adi dan bg pandji serti publik yg ada di k...,True
2,@BagasRyan-w5q,Kelas mas adi prayitno memang benar yg kalah g...,2025-03-17T19:06:15Z,0,Kelas mas adi prayitno memang benar yg kalah g...,True
3,@sahrulmeltas7551,Dari sini kita faham bahwa parpol nya tidak ma...,2025-03-14T07:38:22Z,0,Dari sini kita faham bahwa parpol nya tidak ma...,True
4,@SadegaTri,"Seneng lihat acara ini, tapi sedikit masih sda...",2025-03-09T18:48:53Z,0,"Seneng lihat acara ini, tapi sedikit masih sda...",True
...,...,...,...,...,...,...
18587,@whoami4767,Oh tempe,2025-03-22T00:54:53Z,5,Oh tempe,True
18588,@firmantuhan-k9o,"TIM BOCOR ALUS JANGAN TAKUT, UNTUK KEBENARAN T...",2025-03-22T00:34:47Z,46,"TIM BOCOR ALUS JANGAN TAKUT, UNTUK KEBENARAN T...",True
18589,@Local_indosuke2004,Presiden beban,2025-03-22T00:22:54Z,23,Presiden beban,True
18590,@andipamungkas3397,"Semangat terus tempo, jangan kalah sama teror ...",2025-03-21T23:39:37Z,96,"Semangat terus tempo, jangan kalah sama teror ...",True


In [59]:
data_clean = data.drop(columns=['Komentar', 'Likes', 'Public'])
data_clean

,Username,Timestamp,Original Text
0,@blackberry1986,2025-03-25T03:07:32Z,"Coba dpr kalau mau buat undang"" bikin vote rak..."
1,@UbaidillahAlhasani,2025-03-19T21:29:18Z,mas adi dan bg pandji serti publik yg ada di k...
2,@BagasRyan-w5q,2025-03-17T19:06:15Z,Kelas mas adi prayitno memang benar yg kalah g...
3,@sahrulmeltas7551,2025-03-14T07:38:22Z,Dari sini kita faham bahwa parpol nya tidak ma...
4,@SadegaTri,2025-03-09T18:48:53Z,"Seneng lihat acara ini, tapi sedikit masih sda..."
...,...,...,...
18587,@whoami4767,2025-03-22T00:54:53Z,Oh tempe
18588,@firmantuhan-k9o,2025-03-22T00:34:47Z,"TIM BOCOR ALUS JANGAN TAKUT, UNTUK KEBENARAN T..."
18589,@Local_indosuke2004,2025-03-22T00:22:54Z,Presiden beban
18590,@andipamungkas3397,2025-03-21T23:39:37Z,"Semangat terus tempo, jangan kalah sama teror ..."


In [60]:
# drop duplicates
data_clean = data_clean.drop_duplicates()
data_clean

,Username,Timestamp,Original Text
0,@blackberry1986,2025-03-25T03:07:32Z,"Coba dpr kalau mau buat undang"" bikin vote rak..."
1,@UbaidillahAlhasani,2025-03-19T21:29:18Z,mas adi dan bg pandji serti publik yg ada di k...
2,@BagasRyan-w5q,2025-03-17T19:06:15Z,Kelas mas adi prayitno memang benar yg kalah g...
3,@sahrulmeltas7551,2025-03-14T07:38:22Z,Dari sini kita faham bahwa parpol nya tidak ma...
4,@SadegaTri,2025-03-09T18:48:53Z,"Seneng lihat acara ini, tapi sedikit masih sda..."
...,...,...,...
18587,@whoami4767,2025-03-22T00:54:53Z,Oh tempe
18588,@firmantuhan-k9o,2025-03-22T00:34:47Z,"TIM BOCOR ALUS JANGAN TAKUT, UNTUK KEBENARAN T..."
18589,@Local_indosuke2004,2025-03-22T00:22:54Z,Presiden beban
18590,@andipamungkas3397,2025-03-21T23:39:37Z,"Semangat terus tempo, jangan kalah sama teror ..."


In [61]:
# drop missing values
data_clean = data_clean.dropna()
data_clean

,Username,Timestamp,Original Text
0,@blackberry1986,2025-03-25T03:07:32Z,"Coba dpr kalau mau buat undang"" bikin vote rak..."
1,@UbaidillahAlhasani,2025-03-19T21:29:18Z,mas adi dan bg pandji serti publik yg ada di k...
2,@BagasRyan-w5q,2025-03-17T19:06:15Z,Kelas mas adi prayitno memang benar yg kalah g...
3,@sahrulmeltas7551,2025-03-14T07:38:22Z,Dari sini kita faham bahwa parpol nya tidak ma...
4,@SadegaTri,2025-03-09T18:48:53Z,"Seneng lihat acara ini, tapi sedikit masih sda..."
...,...,...,...
18587,@whoami4767,2025-03-22T00:54:53Z,Oh tempe
18588,@firmantuhan-k9o,2025-03-22T00:34:47Z,"TIM BOCOR ALUS JANGAN TAKUT, UNTUK KEBENARAN T..."
18589,@Local_indosuke2004,2025-03-22T00:22:54Z,Presiden beban
18590,@andipamungkas3397,2025-03-21T23:39:37Z,"Semangat terus tempo, jangan kalah sama teror ..."


In [62]:
def clean_text(text):
    # Menghapus tautan
    text = re.sub(r'http[s]?://\S+', '', text)
    # Menghapus kalimat yang sama dengan tautan yang dihapus
    text = re.sub(r'\b\w+\b(?:\s+\b\w+\b)*\s*http[s]?://\S+', '', text)
    # Menghapus tanda baca
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Menghapus spasi ekstra
    text = ' '.join(text.split())
    # Konversi emoji menjadi teks
    text = emoji.demojize(text, delimiters=(' ', ''))
    return text

# Terapkan fungsi pembersihan ke kolom 'Original Text'
data_clean['Cleaned Text'] = data_clean['Original Text'].apply(clean_text)

C:\Users\Yoan Rifqi\AppData\Local\Temp\ipykernel_11372\1289146322.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_clean['Cleaned Text'] = data_clean['Original Text'].apply(clean_text)


In [63]:
# case folding
data_clean['Lower Text'] = data_clean['Cleaned Text'].str.lower()
data_clean

C:\Users\Yoan Rifqi\AppData\Local\Temp\ipykernel_11372\557343877.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_clean['Lower Text'] = data_clean['Cleaned Text'].str.lower()


,Username,Timestamp,Original Text,Cleaned Text,Lower Text
0,@blackberry1986,2025-03-25T03:07:32Z,"Coba dpr kalau mau buat undang"" bikin vote rak...",Coba dpr kalau mau buat undang bikin vote raky...,coba dpr kalau mau buat undang bikin vote raky...
1,@UbaidillahAlhasani,2025-03-19T21:29:18Z,mas adi dan bg pandji serti publik yg ada di k...,mas adi dan bg pandji serti publik yg ada di k...,mas adi dan bg pandji serti publik yg ada di k...
2,@BagasRyan-w5q,2025-03-17T19:06:15Z,Kelas mas adi prayitno memang benar yg kalah g...,Kelas mas adi prayitno memang benar yg kalah g...,kelas mas adi prayitno memang benar yg kalah g...
3,@sahrulmeltas7551,2025-03-14T07:38:22Z,Dari sini kita faham bahwa parpol nya tidak ma...,Dari sini kita faham bahwa parpol nya tidak ma...,dari sini kita faham bahwa parpol nya tidak ma...
4,@SadegaTri,2025-03-09T18:48:53Z,"Seneng lihat acara ini, tapi sedikit masih sda...",Seneng lihat acara ini tapi sedikit masih sda ...,seneng lihat acara ini tapi sedikit masih sda ...
...,...,...,...,...,...
18587,@whoami4767,2025-03-22T00:54:53Z,Oh tempe,Oh tempe,oh tempe
18588,@firmantuhan-k9o,2025-03-22T00:34:47Z,"TIM BOCOR ALUS JANGAN TAKUT, UNTUK KEBENARAN T...",TIM BOCOR ALUS JANGAN TAKUT UNTUK KEBENARAN TU...,tim bocor alus jangan takut untuk kebenaran tu...
18589,@Local_indosuke2004,2025-03-22T00:22:54Z,Presiden beban,Presiden beban,presiden beban
18590,@andipamungkas3397,2025-03-21T23:39:37Z,"Semangat terus tempo, jangan kalah sama teror ...",Semangat terus tempo jangan kalah sama teror babi,semangat terus tempo jangan kalah sama teror babi


In [64]:
# kamus normalisasi
normalization_dict = {
    "gk": "tidak",
    "ga": "tidak",
    "nggak": "tidak",
    "enggak": "tidak",
    "gak": "tidak",
    "sy": "saya",
    "saya": "saya",
    "gw": "saya",
    "gue": "saya",
    "gua": "saya",
    "ane": "saya",
    "bgt": "banget",
    "bgd": "banget",
    "banget": "banget",
    "sng": "sangat",
    "aja": "saja",
    "bro": "teman",
    "sist": "teman",
    "brother": "teman",
    "sis": "teman",
    "temen": "teman",
    "dgn": "dengan",
    "dg": "dengan",
    "kmrn": "kemarin",
    "kmren": "kemarin",
    "besok": "besok",
    "besokk": "besok",
    "mending": "lebih baik",
    "bangettt": "banget",
    "dr": "dari",
    "drpd": "daripada",
    "spt": "seperti",
    "sama": "dengan",
    "org": "orang",
    "om": "paman",
    "tante": "bibi",
    "bgtu": "begitu",
    "lha": "saya",
    "pd": "pada",
    "buat": "untuk",
    "untk": "untuk",
    "utk": "untuk",
    "kayak": "seperti",
    "yg": "yang",
    "aja": "saja",
    "byk": "banyak",
    "bnyk": "banyak",
    "jd": "jadi",
    "jdii": "jadi",
    "ngapain": "kenapa",
    "knp": "kenapa",
    "dmn": "dimana",
    "dimn": "dimana",
    "blg": "bilang",
    "tau": "tahu",
    "tauu": "tahu",
    "tauh": "tahu",
    "ngga": "tidak",
    "blm": "belum",
    "belom": "belum",
    "tdk": "tidak",
    "trs": "terus",
    "sm": "sama",
    "bs": "bisa",
    "gpp": "tidak apa-apa",
    "oke": "baik",
    "ok": "baik",
    "sip": "baik",
    "okey": "baik",
    "skrg": "sekarang",
    "skg": "sekarang",
    "sdh": "sudah",
    "udh": "sudah",
    "mau": "ingin",
    "ckp": "cukup",
    "dkt": "dekat",
    "deket": "dekat",
    "pas": "ketika",
    "thn": "tahun",
    "th": "tahun",
    "pls": "tolong",
    "tolong": "tolong",
    "pliss": "tolong",
    "thx": "terima kasih",
    "thanks": "terima kasih",
    "makasih": "terima kasih",
    "makasihh": "terima kasih",
    "terimakasih": "terima kasih",
    "tnpa": "tanpa",
    "btw": "omong-omong",
    "nggk": "tidak",
    "nggk": "tidak",
    "lg": "lagi",
    "lagi": "lagi",
    "blg": "bilang",
    "cm": "cuma",
    "cuman": "hanya",
    "cmn": "cuma",
    "liat": "lihat",
    "kmn": "kemana",
    "kalo" : "kalau",
    "cmn": "cuma",
    "nanya": "bertanya",
    "nyuruh": "menyuruh",
    "bsok": "besok",
    "td": "tadi",
    "trus": "terus",
    "bljr": "belajar",
    "sbk": "sibuk",
    "dg": "dengan",
    "nyoba": "mencoba",
    "dlu": "dulu",
    "mslh": "masalah",
    "mslh": "masalah",
    "semngat": "semangat",
    "pake": "menggunakan",
    "gitu": "seperti itu",
    "gituan": "seperti itu",
    "gimana": "bagaimana",
    "yg": "yang",
    "aja": "saja",
    "kl": "kalau",
    "klu": "kalau",
    "tlg": "tolong",
    "btw": "omong-omong"
}

# normalisasi teks
def normalize_text(text, normalization_dict):
    words = text.split()
    normalized_words = [normalization_dict.get(word, word) for word in words]
    return ' '.join(normalized_words)

data_clean['Normalize Text'] = data_clean['Lower Text'].apply(lambda x: normalize_text(x, normalization_dict))

C:\Users\Yoan Rifqi\AppData\Local\Temp\ipykernel_11372\2829855886.py:136: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_clean['Normalize Text'] = data_clean['Lower Text'].apply(lambda x: normalize_text(x, normalization_dict))


In [65]:
data_clean

,Username,Timestamp,Original Text,Cleaned Text,Lower Text,Normalize Text
0,@blackberry1986,2025-03-25T03:07:32Z,"Coba dpr kalau mau buat undang"" bikin vote rak...",Coba dpr kalau mau buat undang bikin vote raky...,coba dpr kalau mau buat undang bikin vote raky...,coba dpr kalau ingin untuk undang bikin vote r...
1,@UbaidillahAlhasani,2025-03-19T21:29:18Z,mas adi dan bg pandji serti publik yg ada di k...,mas adi dan bg pandji serti publik yg ada di k...,mas adi dan bg pandji serti publik yg ada di k...,mas adi dan bg pandji serti publik yang ada di...
2,@BagasRyan-w5q,2025-03-17T19:06:15Z,Kelas mas adi prayitno memang benar yg kalah g...,Kelas mas adi prayitno memang benar yg kalah g...,kelas mas adi prayitno memang benar yg kalah g...,kelas mas adi prayitno memang benar yang kalah...
3,@sahrulmeltas7551,2025-03-14T07:38:22Z,Dari sini kita faham bahwa parpol nya tidak ma...,Dari sini kita faham bahwa parpol nya tidak ma...,dari sini kita faham bahwa parpol nya tidak ma...,dari sini kita faham bahwa parpol nya tidak in...
4,@SadegaTri,2025-03-09T18:48:53Z,"Seneng lihat acara ini, tapi sedikit masih sda...",Seneng lihat acara ini tapi sedikit masih sda ...,seneng lihat acara ini tapi sedikit masih sda ...,seneng lihat acara ini tapi sedikit masih sda ...
...,...,...,...,...,...,...
18587,@whoami4767,2025-03-22T00:54:53Z,Oh tempe,Oh tempe,oh tempe,oh tempe
18588,@firmantuhan-k9o,2025-03-22T00:34:47Z,"TIM BOCOR ALUS JANGAN TAKUT, UNTUK KEBENARAN T...",TIM BOCOR ALUS JANGAN TAKUT UNTUK KEBENARAN TU...,tim bocor alus jangan takut untuk kebenaran tu...,tim bocor alus jangan takut untuk kebenaran tu...
18589,@Local_indosuke2004,2025-03-22T00:22:54Z,Presiden beban,Presiden beban,presiden beban,presiden beban
18590,@andipamungkas3397,2025-03-21T23:39:37Z,"Semangat terus tempo, jangan kalah sama teror ...",Semangat terus tempo jangan kalah sama teror babi,semangat terus tempo jangan kalah sama teror babi,semangat terus tempo jangan kalah dengan teror...


In [66]:
# Inisialisasi factory stopword remover
factory = StopWordRemoverFactory()
stopwords_remover = factory.create_stop_word_remover()

def stopword_remove_sastrawi(text):
    # Menghapus stopword menggunakan Sastrawi
    cleaned_text = stopwords_remover.remove(text)
    
    # Menghapus kata 'blackpink' dan 'bri' (case-insensitive dan hanya kata lengkap)
    cleaned_text = re.sub(r'\b(?:blackpink|bri)\b', '', cleaned_text, flags=re.IGNORECASE)
    
    # Menghapus spasi berlebih
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text).strip()
    
    return cleaned_text

# Terapkan pada kolom 'Normalize Text'
data_clean['Stopwords Text'] = data_clean['Normalize Text'].apply(stopword_remove_sastrawi)

C:\Users\Yoan Rifqi\AppData\Local\Temp\ipykernel_11372\1184672428.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_clean['Stopwords Text'] = data_clean['Normalize Text'].apply(stopword_remove_sastrawi)


In [67]:
data_clean

,Username,Timestamp,Original Text,Cleaned Text,Lower Text,Normalize Text,Stopwords Text
0,@blackberry1986,2025-03-25T03:07:32Z,"Coba dpr kalau mau buat undang"" bikin vote rak...",Coba dpr kalau mau buat undang bikin vote raky...,coba dpr kalau mau buat undang bikin vote raky...,coba dpr kalau ingin untuk undang bikin vote r...,coba dpr kalau untuk undang bikin vote rakyat ...
1,@UbaidillahAlhasani,2025-03-19T21:29:18Z,mas adi dan bg pandji serti publik yg ada di k...,mas adi dan bg pandji serti publik yg ada di k...,mas adi dan bg pandji serti publik yg ada di k...,mas adi dan bg pandji serti publik yang ada di...,mas adi bg pandji serti publik ada kolom komen...
2,@BagasRyan-w5q,2025-03-17T19:06:15Z,Kelas mas adi prayitno memang benar yg kalah g...,Kelas mas adi prayitno memang benar yg kalah g...,kelas mas adi prayitno memang benar yg kalah g...,kelas mas adi prayitno memang benar yang kalah...,kelas mas adi prayitno memang benar kalah perl...
3,@sahrulmeltas7551,2025-03-14T07:38:22Z,Dari sini kita faham bahwa parpol nya tidak ma...,Dari sini kita faham bahwa parpol nya tidak ma...,dari sini kita faham bahwa parpol nya tidak ma...,dari sini kita faham bahwa parpol nya tidak in...,sini faham parpol nya ingin kritik
4,@SadegaTri,2025-03-09T18:48:53Z,"Seneng lihat acara ini, tapi sedikit masih sda...",Seneng lihat acara ini tapi sedikit masih sda ...,seneng lihat acara ini tapi sedikit masih sda ...,seneng lihat acara ini tapi sedikit masih sda ...,seneng lihat acara tapi sedikit sda kurang fai...
...,...,...,...,...,...,...,...
18587,@whoami4767,2025-03-22T00:54:53Z,Oh tempe,Oh tempe,oh tempe,oh tempe,tempe
18588,@firmantuhan-k9o,2025-03-22T00:34:47Z,"TIM BOCOR ALUS JANGAN TAKUT, UNTUK KEBENARAN T...",TIM BOCOR ALUS JANGAN TAKUT UNTUK KEBENARAN TU...,tim bocor alus jangan takut untuk kebenaran tu...,tim bocor alus jangan takut untuk kebenaran tu...,tim bocor alus jangan takut kebenaran tuhan me...
18589,@Local_indosuke2004,2025-03-22T00:22:54Z,Presiden beban,Presiden beban,presiden beban,presiden beban,presiden beban
18590,@andipamungkas3397,2025-03-21T23:39:37Z,"Semangat terus tempo, jangan kalah sama teror ...",Semangat terus tempo jangan kalah sama teror babi,semangat terus tempo jangan kalah sama teror babi,semangat terus tempo jangan kalah dengan teror...,semangat terus tempo jangan kalah teror babi


In [70]:
from nltk.corpus import stopwords

stop_words = set(stopwords.words('Indonesian'))

def remove_stopwords(text):
    words = text.split()
    filtered_words = [word for word in words if word.lower() not in stop_words]
    return ' '.join(filtered_words)

data_clean['Stopwords Text'] = data_clean['Stopwords Text'].apply(remove_stopwords)

C:\Users\Yoan Rifqi\AppData\Local\Temp\ipykernel_11372\1868454159.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_clean['Stopwords Text'] = data_clean['Stopwords Text'].apply(remove_stopwords)


In [71]:
data_clean

,Username,Timestamp,Original Text,Cleaned Text,Lower Text,Normalize Text,Stopwords Text
0,@blackberry1986,2025-03-25T03:07:32Z,"Coba dpr kalau mau buat undang"" bikin vote rak...",Coba dpr kalau mau buat undang bikin vote raky...,coba dpr kalau mau buat undang bikin vote raky...,coba dpr kalau ingin untuk undang bikin vote r...,coba dpr undang bikin vote rakyat biar rakyat ...
1,@UbaidillahAlhasani,2025-03-19T21:29:18Z,mas adi dan bg pandji serti publik yg ada di k...,mas adi dan bg pandji serti publik yg ada di k...,mas adi dan bg pandji serti publik yg ada di k...,mas adi dan bg pandji serti publik yang ada di...,mas adi bg pandji serti publik kolom komentar buk
2,@BagasRyan-w5q,2025-03-17T19:06:15Z,Kelas mas adi prayitno memang benar yg kalah g...,Kelas mas adi prayitno memang benar yg kalah g...,kelas mas adi prayitno memang benar yg kalah g...,kelas mas adi prayitno memang benar yang kalah...,kelas mas adi prayitno kalah cawecawe mendukun...
3,@sahrulmeltas7551,2025-03-14T07:38:22Z,Dari sini kita faham bahwa parpol nya tidak ma...,Dari sini kita faham bahwa parpol nya tidak ma...,dari sini kita faham bahwa parpol nya tidak ma...,dari sini kita faham bahwa parpol nya tidak in...,faham parpol nya kritik
4,@SadegaTri,2025-03-09T18:48:53Z,"Seneng lihat acara ini, tapi sedikit masih sda...",Seneng lihat acara ini tapi sedikit masih sda ...,seneng lihat acara ini tapi sedikit masih sda ...,seneng lihat acara ini tapi sedikit masih sda ...,seneng lihat acara sda fair sih cerminan wakil...
...,...,...,...,...,...,...,...
18587,@whoami4767,2025-03-22T00:54:53Z,Oh tempe,Oh tempe,oh tempe,oh tempe,tempe
18588,@firmantuhan-k9o,2025-03-22T00:34:47Z,"TIM BOCOR ALUS JANGAN TAKUT, UNTUK KEBENARAN T...",TIM BOCOR ALUS JANGAN TAKUT UNTUK KEBENARAN TU...,tim bocor alus jangan takut untuk kebenaran tu...,tim bocor alus jangan takut untuk kebenaran tu...,tim bocor alus takut kebenaran tuhan menyertai
18589,@Local_indosuke2004,2025-03-22T00:22:54Z,Presiden beban,Presiden beban,presiden beban,presiden beban,presiden beban
18590,@andipamungkas3397,2025-03-21T23:39:37Z,"Semangat terus tempo, jangan kalah sama teror ...",Semangat terus tempo jangan kalah sama teror babi,semangat terus tempo jangan kalah sama teror babi,semangat terus tempo jangan kalah dengan teror...,semangat tempo kalah teror babi


In [77]:
# Tokenisasi teks
nlp = spacy.load("xx_ent_wiki_sm")

data_clean['Tokenisasi Text'] = data_clean['Stopwords Text'].apply(lambda x: [token.text for token in nlp(x)])

data_clean

C:\Users\Yoan Rifqi\AppData\Local\Temp\ipykernel_11372\2356363128.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_clean['Tokenisasi Text'] = data_clean['Stopwords Text'].apply(lambda x: [token.text for token in nlp(x)])


,Username,Timestamp,Original Text,Cleaned Text,Lower Text,Normalize Text,Stopwords Text,Tokenisasi Text
0,@blackberry1986,2025-03-25T03:07:32Z,"Coba dpr kalau mau buat undang"" bikin vote rak...",Coba dpr kalau mau buat undang bikin vote raky...,coba dpr kalau mau buat undang bikin vote raky...,coba dpr kalau ingin untuk undang bikin vote r...,coba dpr undang bikin vote rakyat biar rakyat ...,"[coba, dpr, undang, bikin, vote, rakyat, biar,..."
1,@UbaidillahAlhasani,2025-03-19T21:29:18Z,mas adi dan bg pandji serti publik yg ada di k...,mas adi dan bg pandji serti publik yg ada di k...,mas adi dan bg pandji serti publik yg ada di k...,mas adi dan bg pandji serti publik yang ada di...,mas adi bg pandji serti publik kolom komentar buk,"[mas, adi, bg, pandji, serti, publik, kolom, k..."
2,@BagasRyan-w5q,2025-03-17T19:06:15Z,Kelas mas adi prayitno memang benar yg kalah g...,Kelas mas adi prayitno memang benar yg kalah g...,kelas mas adi prayitno memang benar yg kalah g...,kelas mas adi prayitno memang benar yang kalah...,kelas mas adi prayitno kalah cawecawe mendukun...,"[kelas, mas, adi, prayitno, kalah, cawecawe, m..."
3,@sahrulmeltas7551,2025-03-14T07:38:22Z,Dari sini kita faham bahwa parpol nya tidak ma...,Dari sini kita faham bahwa parpol nya tidak ma...,dari sini kita faham bahwa parpol nya tidak ma...,dari sini kita faham bahwa parpol nya tidak in...,faham parpol nya kritik,"[faham, parpol, nya, kritik]"
4,@SadegaTri,2025-03-09T18:48:53Z,"Seneng lihat acara ini, tapi sedikit masih sda...",Seneng lihat acara ini tapi sedikit masih sda ...,seneng lihat acara ini tapi sedikit masih sda ...,seneng lihat acara ini tapi sedikit masih sda ...,seneng lihat acara sda fair sih cerminan wakil...,"[seneng, lihat, acara, sda, fair, sih, cermina..."
...,...,...,...,...,...,...,...,...
18587,@whoami4767,2025-03-22T00:54:53Z,Oh tempe,Oh tempe,oh tempe,oh tempe,tempe,[tempe]
18588,@firmantuhan-k9o,2025-03-22T00:34:47Z,"TIM BOCOR ALUS JANGAN TAKUT, UNTUK KEBENARAN T...",TIM BOCOR ALUS JANGAN TAKUT UNTUK KEBENARAN TU...,tim bocor alus jangan takut untuk kebenaran tu...,tim bocor alus jangan takut untuk kebenaran tu...,tim bocor alus takut kebenaran tuhan menyertai,"[tim, bocor, alus, takut, kebenaran, tuhan, me..."
18589,@Local_indosuke2004,2025-03-22T00:22:54Z,Presiden beban,Presiden beban,presiden beban,presiden beban,presiden beban,"[presiden, beban]"
18590,@andipamungkas3397,2025-03-21T23:39:37Z,"Semangat terus tempo, jangan kalah sama teror ...",Semangat terus tempo jangan kalah sama teror babi,semangat terus tempo jangan kalah sama teror babi,semangat terus tempo jangan kalah dengan teror...,semangat tempo kalah teror babi,"[semangat, tempo, kalah, teror, babi]"
